# 구글드라이브 로드

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 환경설정 keras-lib, requirements set

In [0]:
#env settings
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install -r /content/drive/My\ Drive/Colab\ Notebooks/imports/requirements.txt

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-n5kvk3qb
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-n5kvk3qb
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101065 sha256=7d77b81dcb4a35e8275d73c1a7c94cff80c8e7442013328dd6c6880d44dd5f77
  Stored in directory: /tmp/pip-ephem-wheel-cache-54ry5w5o/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
     |████████████████████████████████| 92.5MB 89kB/s 
     |████████████████████████████████| 2.0MB 31.0MB/s 
     |████████████████████████████████| 17.4MB 503kB/s 
     |████████████████████████████████| 31.2MB 35.4MB/s 
     |████████████████████████████████| 3.2MB 36.3MB/s 
     |████████████████████████████████| 368kB 55.0MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.

# main pix2pix train

! 필요사항
data_loader.py upload
datasets.zip upload

In [36]:
from __future__ import print_function, division
from scipy.misc import imresize

from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.models import model_from_json
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from google.colab import drive, files
import datetime
import matplotlib.pyplot as plt
import sys
from data_loader import DataLoader
import numpy as np
import os
from glob import glob

#! unzip /content/drive/My\ Drive/Colab\ Notebooks/imports/datasets.zip
#! unzip datasets.zip


# os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
class Pix2Pix():
    def __init__(self):
        # Input shape
        self.img_rows = 256
        self.img_cols = 256
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'tmp'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))

        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2 ** 4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        # -------------------------
        # Construct Computational
        #   Graph of Generator
        # -------------------------

        # Build the generator
        self.generator = self.build_generator()

        # Input images and their conditioning images
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # By conditioning on B generate a fake version of A
        fake_A = self.generator(img_B)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminators determines validity of translated images / condition pairs
        valid = self.discriminator([fake_A, img_B])

        self.combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
        self.combined.compile(loss=['mse', 'mae'],
                              loss_weights=[1, 100],
                              optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, bn=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = BatchNormalization(momentum=0.8)(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, bn=False)
        d2 = conv2d(d1, self.gf * 2)
        d3 = conv2d(d2, self.gf * 4)
        d4 = conv2d(d3, self.gf * 8)
        d5 = conv2d(d4, self.gf * 8)
        d6 = conv2d(d5, self.gf * 8)
        d7 = conv2d(d6, self.gf * 8)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf * 8)
        u2 = deconv2d(u1, d5, self.gf * 8)
        u3 = deconv2d(u2, d4, self.gf * 8)
        u4 = deconv2d(u3, d3, self.gf * 4)
        u5 = deconv2d(u4, d2, self.gf * 2)
        u6 = deconv2d(u5, d1, self.gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, bn=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Concatenate image and conditioning image by channels to produce input
        combined_imgs = Concatenate(axis=-1)([img_A, img_B])

        d1 = d_layer(combined_imgs, self.df, bn=False)
        d2 = d_layer(d1, self.df * 2)
        d3 = d_layer(d2, self.df * 4)
        d4 = d_layer(d3, self.df * 8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model([img_A, img_B], validity)

    def train(self, epochs, batch_size=1, sample_interval=50):
        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Condition on B and generate a translated version
                fake_A = self.generator.predict(imgs_B)

                # Train the discriminators (original images = real / generated = Fake)
                d_loss_real = self.discriminator.train_on_batch([imgs_A, imgs_B], valid)
                d_loss_fake = self.discriminator.train_on_batch([fake_A, imgs_B], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # -----------------
                #  Train Generator
                # -----------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, imgs_A])

                elapsed_time = datetime.datetime.now() - start_time

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)

                # Plot the progress
                print("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] time: %s" % (epoch, epochs,
                                                                                                      batch_i,
                                                                                                      self.data_loader.n_batches,
                                                                                                      d_loss[0],
                                                                                                      100 * d_loss[1],
                                                                                                      g_loss[0],
                                                                                                      elapsed_time))

        # serialize model to JSON
        print("Create json")
        com_model_json = self.combined.to_json()
        gen_model_json = self.generator.to_json()
        dis_model_json = self.discriminator.to_json()
        with open("./com_model.json", "w") as json_file:
            json_file.write(com_model_json)
        with open("./gen_model.json", "w") as json_file:
            json_file.write(gen_model_json)
        with open("./dis_model.json", "w") as json_file:
            json_file.write(dis_model_json)            
        # serialize weights to HDF5
        self.combined.save_weights("./com_model.h5")
        self.generator.save_weights("./gen_model.h5")
        self.discriminator.save_weights("./dis_model.h5")
        print("Model saved")

    def sample_images(self, epoch, batch_i):
        # create model
        model = Sequential()
        model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
        model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
        model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 3, 3

        imgs_A, imgs_B = self.data_loader.load_data(batch_size=3, is_testing=True)
        fake_A = self.generator.predict(imgs_B)

        gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Condition', 'Generated', 'Original']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[i])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()


if __name__ == '__main__':
    gan = Pix2Pix()
    counts = input('Train : 1\nTest  : 2\n : ')
    if (int(counts) == 1):
        gan.train(epochs=25, batch_size=1, sample_interval=50)
    if (int(counts) == 2):
        ## use the trained model to generate data
        origin_shape = [[0]*3 for i in range(11)] # 10행 3열 행렬 초기화 
        test_model = gan.generator
        test_model.load_weights("./gen_model.h5")
        path = glob("./datasets/sample/test/*")
        num = 1
        for img in path:
            img_B = scipy.misc.imread(img, mode='RGB').astype(np.float)
            # m, n, d = img_B.shape
            print(img_B.shape)

            origin_shape[num][0] = m
            origin_shape[num][1] = n
            img_B = imresize(img_B, (256, 256, 3))
            print(img_B.shape)

            m, n, d = img_B.shape
            img_show = np.zeros((m, 2 * n, d))
            
            img_b = np.array([img_B]) / 127.5 - 1
            fake_A = 0.5 * (test_model.predict(img_b))[0] + 0.5
            
            img_show[:, :n, :] = img_B / 255
            img_show[:, n:2 * n, :] = fake_A
            scipy.misc.imsave("./%d.jpg" % num,img_show)
            num = num + 1
    """
    files.download('com_model.json')
    files.download('gen_model.json')
    files.download('dis_model.json')
    files.download('com_model.h5')
    files.download('gen_model.h5')
    files.download('dis_model.h5')
    print("finished..")
    """


Train : 1
Test  : 2
 : 2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:260: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:266: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


(32, 70, 3)
(256, 256, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:277: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


(29, 40, 3)
(256, 256, 3)
(49, 86, 3)
(256, 256, 3)
(42, 100, 3)
(256, 256, 3)
(23, 52, 3)
(256, 256, 3)
(23, 50, 3)
(256, 256, 3)
(16, 142, 3)
(256, 256, 3)
(38, 92, 3)
(256, 256, 3)
(17, 240, 3)
(256, 256, 3)
(16, 76, 3)
(256, 256, 3)


# data_loader.py

In [0]:
import scipy
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "test"
        path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs_A = []
        imgs_B = []
        for img_path in batch_images:
            img = self.imread(img_path)

            h, w, _ = img.shape
            _w = int(w/2)
            img_A, img_B = img[:, :_w, :], img[:, _w:, :]

            img_A = scipy.misc.imresize(img_A, self.img_res)
            img_B = scipy.misc.imresize(img_B, self.img_res)

            # If training => do random flip
            if not is_testing and np.random.random() < 0.5:
                img_A = np.fliplr(img_A)
                img_B = np.fliplr(img_B)

            imgs_A.append(img_A)
            imgs_B.append(img_B)

        imgs_A = np.array(imgs_A)/127.5 - 1.
        imgs_B = np.array(imgs_B)/127.5 - 1.

        return imgs_A, imgs_B

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))

        self.n_batches = int(len(path) / batch_size)

        for i in range(self.n_batches-1):
            batch = path[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img in batch:
                img = self.imread(img)
                h, w, _ = img.shape
                half_w = int(w/2)
                img_A = img[:, :half_w, :]
                img_B = img[:, half_w:, :]

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B


    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)


In [0]:
import socketserver
import datetime
import base64
import numpy as np
import cv2
!curl ipecho.net/plain

class MyTCPHandler(socketserver.BaseRequestHandler):

    def handle(self):
        print("get....")
        image1 = []
        try:
            while True:
                data = self.request.recv(82100)  # 클라이언트가보낸데이터를가져옵니다
                print('data,', data)
                # data = base64.b64decode(data)
                if not data or len(data) == 0:
                    break
                image1.extend(data)
            print("get over")
            image = np.asarray(bytearray(image1), dtype="uint8")
            # print("1", image)
            # print("2",len(image))  39559
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            cv2.imwrite("./test.jpg", image)
            cv2.namedWindow("Image")
            cv2.imshow("Image", image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            print("받았습니다")
            self.request.sendall("get your connet!".encode("utf-8"))
        except Exception:
            print(self.client_address, "연결해제")
        finally:
            self.request.close()  # 예외 후 연결을 닫습니다.

    # before handle,연결설정：
    def setup(self):
        now_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print(now_time)
        print("연결설정：", self.client_address)

    # finish run  after handle
    def finish(self):
        print("연결해제")


if __name__ == "__main__":
    HOST, PORT = "", 7777
    # server=socketserver.TCPServer((HOST,PORT),MyTCPHandler)  #매개 변수를 전달하는 인스턴스 객체

    # 멀티스레드
    server = socketserver.ThreadingTCPServer((HOST, PORT), MyTCPHandler)
    server.serve_forever()  # 계속연결

34.69.200.15

KeyboardInterrupt: ignored